The code cell provided below is for the reference code to unzip the unput dataset on your local system.

#### Note: We do not recommend running the code in the lab environment. The zip file size will delay the code execution and may lead to some unforseen errors. The input files have already been unzipped for use in this code.


In [ ]:
# import shutil
# shutil.unpack_archive("Images.zip")

In [ ]:
import tensorflow as tf

# Get the list of available devices
gpus = tf.config.list_physical_devices("GPU")
print("Available GPUs:", gpus)

# # Disable GPU
# if gpus:
#     tf.config.set_visible_devices([], "GPU")
#     print("GPU disabled. Running on CPU.")

# Enable GPU
if gpus:
    tf.config.set_visible_devices(gpus[0], "GPU")
    print("GPU enabled.")

#### Step 1: Data Loading and Preprocessing


In [ ]:
# import pandas,os,cv2 and numpy
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import plotly.express as px

In [ ]:
#### Step 2: Load and preprocess the data

# Load the labels from labels.csv
# // TODO

labels_df = pd.read_csv("./labels.csv")

labels_df.columns = ["image_id", "class", "x_min", "y_min", "x_max", "y_max"]

# Adjust the image IDs in the dataframe
labels_df["image_id"] = labels_df["image_id"].apply(lambda x: f"{x:08d}")

# Use iloc to pick the first 1000 labels
# // TODO
labels_df = labels_df.iloc[:1000]


# Load the corresponding images
images_dir = "Images/"
images = []
for index, row in tqdm(labels_df.iterrows(), total=labels_df.shape[0]):
    img_path = os.path.join(images_dir, f"{row['image_id']}.jpg")
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        images.append(img)
    else:
        print(f"Error loading image: {img_path}")

images = np.array(images)

# Check if images are loaded
# // TODO
print(f"\nNum of Images: {images.shape[0]}")

#### Step 2: Exploratory Data Analysis (EDA)


In [ ]:
# Analyze the distribution of vehicle types in the limited dataset
# // TODO
labels_df["class"].value_counts()

# Address data quality issues arising from the discrepancy between labels and actual image filenames
# Sorting the image filenames
# // TODO
# @ I believe that all of the labels and image file names are correct. I also believe that all of the image file names are sorted.
# @ So if there's something I should do here, please let me know.

#### Step 3: Preprocess the images


In [ ]:
# Check if the 'images' list is not empty
if len(images) > 0:
    # // TODO
    resized_imgs = []
    for img in images:
        # Resize each image in the 'images' list to dimensions 224x224
        resized_imgs.append(cv2.resize(img, (224, 224)))

    # Convert the list of resized images to a NumPy array
    processed_images = np.array(resized_imgs)

    # Print a success message indicating that the images were resized successfully
    print("DONE!")

In [ ]:
processed_images.shape

#### Step4: Prepare the labels and bounding boxes


In [ ]:
labels = labels_df["class"].to_numpy()
bounding_boxes = labels_df[["x_min", "y_min", "x_max", "y_max"]].to_numpy()

# Convert labels to one-hot encoding
unique_labels = np.unique(labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
index_to_label = {index: label for index, label in enumerate(unique_labels)}
labels = np.array([label_to_index[label] for label in labels])

#### Step5: Split the data into training and testing sets


In [ ]:
# // TODO
# Split the data into training and testing sets using the train_test_split function.
# - `processed_images`: The input images that have been resized and preprocessed.
# - `labels`: The corresponding labels for the images.
# - `bounding_boxes`: Bounding box information for the images.
# - `test_size=0.2`: Specifies that 20% of the data will be used for testing, while 80% will be used for training.
# - `random_state=42`: Sets a random seed for reproducibility.
x_train, x_test, y_train, y_test, bbox_train, bbox_test = train_test_split(
    processed_images,
    labels,
    bounding_boxes,
    test_size=0.2,
    random_state=42,
    shuffle=False,
)

#### Step6: Model Creation and Training


In [ ]:
def create_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation="relu")(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    vehicle_class = layers.Dense(
        num_classes, activation="softmax", name="vehicle_class"
    )(x)
    bounding_box = layers.Dense(4, name="bounding_box")(x)

    model = keras.Model(inputs=inputs, outputs=[vehicle_class, bounding_box])
    return model


input_shape = processed_images[0].shape
print(f"input_shape: {input_shape}")
num_classes = len(unique_labels)
print(f"num_classes: {num_classes}")
model = create_model(input_shape, num_classes)


model.compile(
    optimizer="adam",
    loss={"vehicle_class": "sparse_categorical_crossentropy", "bounding_box": "mse"},
    metrics={"vehicle_class": "accuracy", "bounding_box": "mae"},
)

history = model.fit(
    x_train,
    {"vehicle_class": y_train, "bounding_box": bbox_train},
    epochs=25,
    validation_data=(x_test, {"vehicle_class": y_test, "bounding_box": bbox_test}),
)

#### Step7: Model Evaluation


In [ ]:
# Evaluate the model's performance on the test data.
# // TODO
results = model.evaluate(x_test, {"vehicle_class": y_test, "bounding_box": bbox_test})
# The 'model.evaluate' function calculates various metrics and loss values.
# It takes the test data 'x_test' as input and a dictionary that specifies the
# expected outputs for two different tasks: 'vehicle_class' and 'bounding_box'.

# Print the test results to the console.
results

#### Step8: Inferencing and Visualization


In [ ]:
import matplotlib.pyplot as plt

# Choose a few sample images for inference
# // TODO  (Adjust the number of sample images as needed)
sample_images = x_train[0:1]

# Perform inference on the sample images
# // TODO
predictions = model.predict(sample_images)
print(f"predictions: {predictions[1]}")

# Extract the predicted bounding box coordinates
# // TODO
predicted_bounding_boxes = predictions[1]

# Visualize the sample images with predicted bounding boxes
for i in range(len(sample_images)):
    plt.figure()
    plt.imshow(sample_images[i])

    # The red square is where the model guessed where the vehicle is
    plt.gca().add_patch(
        plt.Rectangle(
            (predicted_bounding_boxes[i][0], predicted_bounding_boxes[i][1]),
            predicted_bounding_boxes[i][2] - predicted_bounding_boxes[i][0],
            predicted_bounding_boxes[i][3] - predicted_bounding_boxes[i][1],
            fill=False,
            edgecolor="r",
            linewidth=2,
        )
    )
    # The green square is where the vehicle actually is.
    # (In this case, the Bounding Boxes are wrong because we do not scale the bounding boxes as we do with the images)
    plt.gca().add_patch(
        plt.Rectangle(
            (bbox_train[i][0], bbox_train[i][1]),
            bbox_train[i][2] - bbox_train[i][0],
            bbox_train[i][3] - bbox_train[i][1],
            fill=False,
            edgecolor="g",
            linewidth=2,
        )
    )
    plt.show()